In [1]:
import numpy as np
import sys
import yaml
import random

In [82]:
input_file = 'inputs/expe/input_100ag_1dy_5sta_32_seed1.yaml'
dy_file = 'dy_obstacles/expe/dy_100ag_1dy_5sta_32_seed1.yaml'

In [83]:
N = 100
static_obs_ratio = 0.05
map_size_x = 32
map_size_y = 32
x_min = 0
y_min = 0
seed = 1


num_dynamic_obs = 1
step_num_from = 0
step_num_to = 1
manhadon_dist_min = 3

In [2]:
def write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min):
    input_f = open(input_file, 'a')

    input_dict = {'agents': []}

    start_ls = []
    end_ls = []

    input_f.write('agents:\n')
#     print('agents:')
    for agent_num in range(N):
        while True:
            start_x = random.choice(range(x_min, map_size_x))
            start_y = random.choice(range(y_min, map_size_y))
            end_x = random.choice(range(x_min, map_size_x))
            end_y = random.choice(range(y_min, map_size_y))
            if [start_x, start_y] not in start_ls and [end_x, end_y] not in end_ls:
                start_ls.append([start_x, start_y])
                end_ls.append([end_x, end_y])
                break
            else:
                continue
        input_dict['agents'].append({'start': [start_x, start_y], 'goal': [end_x, end_y], 'name': "agent" + str(agent_num)})
        input_f.write("-   start: " + str([start_x, start_y]) + '\n')
        input_f.write("    goal: " + str([end_x, end_y]) + '\n')
        input_f.write("    name: agent" + str(agent_num) + '\n')
#         print("-   start: ", [start_x, start_y])
#         print("    goal: ", [end_x, end_y])
#         print("    name: agent" + str(agent_num))
        
        
        
        
        
        
        
    num_static_obs = int(static_obs_ratio * (map_size_x * map_size_y))
    # num_static_obs = 3
    obs_ls = []
    input_f.write('map:\n')
    input_f.write('    dimensions: [' + str(map_size_x) + ', ' + str(map_size_y) + ']\n')
    input_f.write('    obstacles:' + '\n')
#     print('map:')
#     print('    dimensions: [', map_size_x, ', ', map_size_y, ']')
#     print('    obstacles:')
    for i in range(num_static_obs):
        while True:
            obs_x = random.choice(range(map_size_x))
            obs_y = random.choice(range(map_size_y))
            if [obs_x, obs_y] not in start_ls and [obs_x, obs_y] not in end_ls and [obs_x, obs_y] not in obs_ls:
                obs_ls.append([obs_x, obs_y])
                break
            else:
                continue
        input_f.write('    - !!python/tuple ' + str([obs_x, obs_y]) + '\n')
#         print('    - !!python/tuple ', [obs_x, obs_y])


        
        
        
        
    input_f.write('obstacles:\n')
    for i in range(num_dynamic_obs):
        input_f.write('-   start: [0, 0]\n')
        input_f.write('    goal: [32, 32]\n')
        input_f.write('    name: obstacle' + str(i) + '\n')

    input_f.close()

    dy_f = open(dy_file, 'a')

#     random.seed(seed)

    dy_f.write('schedule:\n')
#     print('schedule:')
    for dyn_obs in range(num_dynamic_obs):
        dy_f.write('  obstacle' + str(dyn_obs) + ':\n')
#         print('  obstacle' + str(dyn_obs) + ':')
        while True:
            step_x = random.choice(range(map_size_x))
            step_y = random.choice(range(map_size_y))
            step = [step_x, step_y]

            is_bigger = True
            for start_pt in start_ls:
                manhadon_dist = abs(start_pt[0] - step_x) + abs(start_pt[1] - step_y)
                if manhadon_dist < manhadon_dist_min:
                    is_bigger = False
                    break

            if is_bigger:
                if step not in start_ls and step not in end_ls and step not in obs_ls:
                    dy_f.write('  - t: ' + str(0) + '\n')
                    dy_f.write('    x: ' + str(step_x) + '\n')
                    dy_f.write('    y: ' + str(step_y) + '\n')
#                     print('  - t: ', 0)
#                     print('    x: ', step_x)
#                     print('    y: ', step_y)
                    break
        step_num = random.choice(range(step_num_from, step_num_to))
        for step_ in range(step_num):
            while True:
                direction = random.choice(range(5)) # 0 => up | 1 => down | 2 => left | 3 => right | 4 => still 
                if direction == 0:
                    step_x_ = step_x
                    step_y_ = step_y + 1 
                elif direction == 1:
                    step_x_ = step_x
                    step_y_ = step_y - 1 
                elif direction == 2:
                    step_y_ = step_y
                    step_x_ = step_x - 1 
                elif direction == 3:
                    step_y_ = step_y
                    step_x_ = step_x + 1 
                else:
                    step_y_ = step_y
                    step_x_ = step_x

                step = [step_x_, step_y_]

                if step not in end_ls and step not in obs_ls and (0 <= step_x_ < map_size_x) and (0 <= step_y_ < map_size_y):
                    dy_f.write('  - t: ' + str((step_ + 1)) + '\n')
                    dy_f.write('    x: ' + str(step_x) + '\n')
                    dy_f.write('    y: ' + str(step_y) + '\n')
#                     print('  - t: ', (step_ + 1))
#                     print('    x: ', step_x_)
#                     print('    y: ', step_y_)
                    step_x = step_x_
                    step_y = step_y_
                    break
    dy_f.close()

In [30]:
import os
import random
N = 100
static_obs_ratio = 0.05
map_size_x = 32
map_size_y = 32
x_min = 0
y_min = 0
seed = 1

num_dynamic_obs = 1
step_num_from = 0
step_num_to = 1
manhadon_dist_min = 3

tag_initial = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
    + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x)
    
os.mkdir('inputs/table/' + tag_initial)
os.mkdir('outputs/table/' + tag_initial)
os.mkdir('dy_obstacles/table/' + tag_initial)

shell_root = './shells/'
sh_f = open(shell_root + tag_initial + '.sh', 'a')
sh_f.write('cd ../ \n')
for seed in range(20):
    random.seed(seed)

    tag = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
        + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_seed' + str(seed)
    
    input_file = 'inputs/table/' + tag_initial + '/input_' + tag + '.yaml'
    dy_file = 'dy_obstacles/table/' + tag_initial + '/dy_' + tag + '.yaml'
    
    write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min)
    
    sh_f.write('python3 multi_thread_mapf.py inputs/table/' + tag_initial + '/input_' + tag + '.yaml ' + \
              'outputs/table/' + tag_initial + '/output_' + tag + '.yaml ' + \
              'dy_obstacles/table/' + tag_initial + '/dy_' + tag + '.yaml ' + \
              'results/results_' + tag_initial + '.txt \n')
    
sh_f.close()
    

In [8]:
# CBS
import os
import random
N = 50
static_obs_ratio = 0.05
map_size_x = 16
map_size_y = 16
x_min = 0
y_min = 0
seed = 1

num_dynamic_obs = 1
step_num_from = 0
step_num_to = 1
manhadon_dist_min = 3

tag_initial = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
    + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x)
    
# os.mkdir('inputs/table/' + tag_initial)
# os.mkdir('outputs/table/' + tag_initial)
# os.mkdir('dy_obstacles/table/' + tag_initial)

shell_root = './shells/'
sh_f = open(shell_root + tag_initial + '.sh', 'a')
sh_f.write('cd ../ \n')
for seed in range(20):
    random.seed(seed)

    tag = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
        + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_seed' + str(seed)
    
    input_file = 'inputs/table/' + tag_initial + '/input_' + tag + '.yaml'
    dy_file = 'dy_obstacles/table/' + tag_initial + '/dy_' + tag + '.yaml'
    
#     write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min)
    
    sh_f.write('python3 cbs.py inputs/table/' + tag_initial + '/input_' + tag + '.yaml ' + \
              'outputs/table/' + tag_initial + '/output_' + tag + '.yaml ' + \
              'results/results_' + tag_initial + '.txt \n')
    
sh_f.close()
    

In [10]:
# CBS
import os
import random

for N in range(10, 110, 10):
    for static_obs_ratio in [0.01, 0.05, 0.1]:
        map_size_x = 32
        map_size_y = 32
        x_min = 0
        y_min = 0
        seed = 1

        num_dynamic_obs = 1
        step_num_from = 0
        step_num_to = 1
        manhadon_dist_min = 3

        tag_initial = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
            + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x)

        os.mkdir('inputs/table/' + tag_initial)
        os.mkdir('outputs/table/' + tag_initial)
        os.mkdir('dy_obstacles/table/' + tag_initial)

        shell_root = './shells/'
        sh_f = open(shell_root + tag_initial + '.sh', 'a')
        sh_f.write('cd ../ \n')
        for seed in range(20):
            random.seed(seed)

            tag = str(N) + 'ag_' + str(num_dynamic_obs) + 'dy_' \
                + str(int(static_obs_ratio * 100)) + 'sta_' + str(map_size_x) + '_seed' + str(seed)

            input_file = 'inputs/table/' + tag_initial + '/input_' + tag + '.yaml'
            dy_file = 'dy_obstacles/table/' + tag_initial + '/dy_' + tag + '.yaml'

            write_all(input_file, dy_file, N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min, num_dynamic_obs, step_num_from, step_num_to, manhadon_dist_min)

            sh_f.write('python3 cbs.py inputs/table/' + tag_initial + '/input_' + tag + '.yaml ' + \
                      'outputs/table/' + tag_initial + '/output_' + tag + '.yaml ' + \
                      'results/results_' + tag_initial + '.txt \n')

        sh_f.close()


In [5]:
 ./10ag_1dy_10sta_32.sh  && ./10ag_1dy_1sta_32.sh  && ./10ag_1dy_5sta_32.sh  && ./20ag_1dy_10sta_32.sh

10
20
30
40
50
60
70
80
90
100


In [84]:
random.seed(seed)
# def write_input(N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min):
input_f = open(input_file, 'a')

input_dict = {'agents': []}

start_ls = []
end_ls = []

input_f.write('agents:\n')
print('agents:')
for agent_num in range(N):
    while True:
        start_x = random.choice(range(x_min, map_size_x))
        start_y = random.choice(range(y_min, map_size_y))
        end_x = random.choice(range(x_min, map_size_x))
        end_y = random.choice(range(y_min, map_size_y))
        if [start_x, start_y] not in start_ls and [end_x, end_y] not in end_ls:
            start_ls.append([start_x, start_y])
            end_ls.append([end_x, end_y])
            break
        else:
            continue
    input_dict['agents'].append({'start': [start_x, start_y], 'goal': [end_x, end_y], 'name': "agent" + str(agent_num)})
    input_f.write("-   start: " + str([start_x, start_y]) + '\n')
    input_f.write("    goal: " + str([end_x, end_y]) + '\n')
    input_f.write("    name: agent" + str(agent_num) + '\n')
    print("-   start: ", [start_x, start_y])
    print("    goal: ", [end_x, end_y])
    print("    name: agent" + str(agent_num))


# print(start_ls)
# print(end_ls)

agents:
-   start:  [8, 4]
    goal:  [16, 7]
    name: agent0
-   start:  [31, 28]
    goal:  [30, 24]
    name: agent1
-   start:  [13, 6]
    goal:  [31, 1]
    name: agent2
-   start:  [24, 27]
    goal:  [0, 28]
    name: agent3
-   start:  [17, 14]
    goal:  [6, 20]
    name: agent4
-   start:  [1, 1]
    goal:  [1, 0]
    name: agent5
-   start:  [24, 13]
    goal:  [27, 1]
    name: agent6
-   start:  [14, 28]
    goal:  [31, 14]
    name: agent7
-   start:  [22, 14]
    goal:  [14, 29]
    name: agent8
-   start:  [18, 1]
    goal:  [26, 6]
    name: agent9
-   start:  [11, 18]
    goal:  [7, 21]
    name: agent10
-   start:  [27, 12]
    goal:  [19, 18]
    name: agent11
-   start:  [31, 25]
    goal:  [2, 30]
    name: agent12
-   start:  [15, 25]
    goal:  [26, 11]
    name: agent13
-   start:  [23, 23]
    goal:  [5, 28]
    name: agent14
-   start:  [6, 10]
    goal:  [25, 23]
    name: agent15
-   start:  [31, 1]
    goal:  [30, 2]
    name: agent16
-   start:  [19, 25

In [85]:
# def write_sta_obs(N, static_obs_ratio, map_size_x, map_size_y, x_min, y_min):
num_static_obs = int(static_obs_ratio * (map_size_x * map_size_y))
# num_static_obs = 3
obs_ls = []
input_f.write('map:\n')
input_f.write('    dimensions: [' + str(map_size_x) + ', ' + str(map_size_y) + ']\n')
input_f.write('    obstacles:' + '\n')
print('map:')
print('    dimensions: [', map_size_x, ', ', map_size_y, ']')
print('    obstacles:')
for i in range(num_static_obs):
    while True:
        obs_x = random.choice(range(map_size_x))
        obs_y = random.choice(range(map_size_y))
        if [obs_x, obs_y] not in start_ls and [obs_x, obs_y] not in end_ls and [obs_x, obs_y] not in obs_ls:
            obs_ls.append([obs_x, obs_y])
            break
        else:
            continue
    input_f.write('    - !!python/tuple ' + str([obs_x, obs_y]) + '\n')
    print('    - !!python/tuple ', [obs_x, obs_y])


# print(obs_ls)

map:
    dimensions: [ 32 ,  32 ]
    obstacles:
    - !!python/tuple  [12, 17]
    - !!python/tuple  [19, 16]
    - !!python/tuple  [22, 31]
    - !!python/tuple  [13, 24]
    - !!python/tuple  [13, 18]
    - !!python/tuple  [6, 1]
    - !!python/tuple  [7, 0]
    - !!python/tuple  [18, 8]
    - !!python/tuple  [19, 27]
    - !!python/tuple  [22, 20]
    - !!python/tuple  [0, 7]
    - !!python/tuple  [28, 28]
    - !!python/tuple  [22, 19]
    - !!python/tuple  [31, 7]
    - !!python/tuple  [24, 24]
    - !!python/tuple  [13, 0]
    - !!python/tuple  [17, 12]
    - !!python/tuple  [29, 26]
    - !!python/tuple  [19, 10]
    - !!python/tuple  [28, 12]
    - !!python/tuple  [23, 0]
    - !!python/tuple  [4, 31]
    - !!python/tuple  [15, 18]
    - !!python/tuple  [1, 26]
    - !!python/tuple  [9, 25]
    - !!python/tuple  [17, 11]
    - !!python/tuple  [4, 0]
    - !!python/tuple  [22, 16]
    - !!python/tuple  [26, 19]
    - !!python/tuple  [9, 29]
    - !!python/tuple  [16, 31]
    - 

In [86]:
input_f.write('obstacles:\n')
for i in range(num_dynamic_obs):
    input_f.write('-   start: [0, 0]\n')
    input_f.write('    goal: [32, 32]\n')
    input_f.write('    name: obstacle' + str(i) + '\n')
        
input_f.close()

dy_f = open(dy_file, 'a')

random.seed(seed)

dy_f.write('schedule:\n')
print('schedule:')
for dyn_obs in range(num_dynamic_obs):
    dy_f.write('  obstacle' + str(dyn_obs) + ':\n')
    print('  obstacle' + str(dyn_obs) + ':')
    while True:
        step_x = random.choice(range(map_size_x))
        step_y = random.choice(range(map_size_y))
        step = [step_x, step_y]
        
        is_bigger = True
        for start_pt in start_ls:
            manhadon_dist = abs(start_pt[0] - step_x) + abs(start_pt[1] - step_y)
            if manhadon_dist < manhadon_dist_min:
                is_bigger = False
                break
                
        if is_bigger:
            if step not in start_ls and step not in end_ls and step not in obs_ls:
                dy_f.write('  - t: ' + str(0) + '\n')
                dy_f.write('    x: ' + str(step_x) + '\n')
                dy_f.write('    y: ' + str(step_y) + '\n')
                print('  - t: ', 0)
                print('    x: ', step_x)
                print('    y: ', step_y)
                break
    step_num = random.choice(range(step_num_from, step_num_to))
    for step_ in range(step_num):
        while True:
            direction = random.choice(range(5)) # 0 => up | 1 => down | 2 => left | 3 => right | 4 => still 
            if direction == 0:
                step_x_ = step_x
                step_y_ = step_y + 1 
            elif direction == 1:
                step_x_ = step_x
                step_y_ = step_y - 1 
            elif direction == 2:
                step_y_ = step_y
                step_x_ = step_x - 1 
            elif direction == 3:
                step_y_ = step_y
                step_x_ = step_x + 1 
            else:
                step_y_ = step_y
                step_x_ = step_x

            step = [step_x_, step_y_]

            if step not in end_ls and step not in obs_ls and (0 <= step_x_ < map_size_x) and (0 <= step_y_ < map_size_y):
                dy_f.write('  - t: ' + str((step_ + 1)) + '\n')
                dy_f.write('    x: ' + str(step_x) + '\n')
                dy_f.write('    y: ' + str(step_y) + '\n')
                print('  - t: ', (step_ + 1))
                print('    x: ', step_x_)
                print('    y: ', step_y_)
                step_x = step_x_
                step_y = step_y_
                break
dy_f.close()

schedule:
  obstacle0:
  - t:  0
    x:  12
    y:  31


In [48]:
with open('inputs/input.yaml', 'r') as file:
    try:
        input_file = yaml.load(file, Loader=yaml.FullLoader)
    except yaml.YAMLError as exc:
        print(exc)
        
start_ls = []
end_ls = []
        
for agent in input_file['agents']:
    start_ls.append(agent['start'])
    end_ls.append(agent['goal'])

In [8]:
input_file

{'agents': [{'start': [30, 17], 'goal': [22, 9], 'name': 'agent0'},
  {'start': [24, 0], 'goal': [17, 29], 'name': 'agent1'},
  {'start': [14, 0], 'goal': [9, 28], 'name': 'agent2'},
  {'start': [23, 10], 'goal': [21, 13], 'name': 'agent3'},
  {'start': [3, 12], 'goal': [25, 5], 'name': 'agent4'},
  {'start': [1, 3], 'goal': [14, 5], 'name': 'agent5'},
  {'start': [27, 28], 'goal': [7, 27], 'name': 'agent6'},
  {'start': [8, 20], 'goal': [10, 5], 'name': 'agent7'},
  {'start': [30, 30], 'goal': [5, 12], 'name': 'agent8'},
  {'start': [16, 22], 'goal': [23, 24], 'name': 'agent9'}],
 'map': {'dimensions': [32, 32],
  'obstacles': [(0, 7),
   (0, 13),
   (0, 22),
   (0, 27),
   (0, 28),
   (0, 29),
   (1, 5),
   (1, 8),
   (1, 11),
   (1, 17),
   (1, 21),
   (1, 23),
   (1, 24),
   (2, 0),
   (2, 1),
   (2, 3),
   (2, 10),
   (2, 14),
   (2, 25),
   (3, 1),
   (3, 5),
   (3, 6),
   (3, 8),
   (3, 9),
   (3, 10),
   (3, 24),
   (4, 6),
   (4, 13),
   (4, 16),
   (4, 18),
   (4, 19),
   (4,